#Importando pacotes

In [ ]:
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import seaborn as sns
sns.set()

# Nova seção

In [ ]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


#Criando uma coluna OMS para identificar se o texto menciona OMS

In [ ]:
STF_STJ = pd.read_excel('/content/mydrive/MyDrive/OMS-Conteudos-STJ_STF.xlsx')
STF_STJ['OMS'] = np.nan
STF_STJ

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,NaN
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,NaN
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,NaN
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,NaN
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,NaN
...,...,...,...,...,...,...,...,...
17415,2022-03-28,RHC,157460,13086443,stj,AgRg no RECURSO EM HABEAS CORPUS Nº 157.460 - ...,Tabaco_Tobacco,NaN
17416,2022-03-30,AREsp,2073755,12259496,stj,AGRAVO EM RECURSO ESPECIAL Nº 2.073.755 - PI (...,Tabaco_Tobacco,NaN
17417,2022-03-31,HC,727811,12545136,stj,AgRg no HABEAS CORPUS Nº 727.811 - RS (2022/00...,Tabaco_Tobacco,NaN
17418,2022-03-31,RHC,161755,13096384,stj,AgRg no RECURSO EM HABEAS CORPUS Nº 161.755 - ...,Tabaco_Tobacco,NaN


#Separando STF e STJ

In [ ]:
STF = STF_STJ.loc[STF_STJ['Origem'] == 'stf']
STF

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,NaN
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,NaN
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,NaN
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,NaN
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,NaN
...,...,...,...,...,...,...,...,...
16518,2021-07-28,HC,203222,15347144135,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,NaN
16519,2021-08-02,HC,203222,15347162829,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,NaN
16520,2021-08-24,AR,2833,15347432581,stf,Ementa e Acórdão\n\n17/08/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,NaN
16521,2021-10-08,RE,1335917,15348181694,stf,Ementa e Acórdão\n\n27/09/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,NaN


#Criando coluna teste para comparar com um dataframe com a palavra OMS

In [ ]:
#criando coluna "teste_PROBABILIDADE"='PROBABILIDADE_OMS'   e   "teste_RESULTADO_df1 "='RESULTADO_OMS'
STF= STF.assign(PROBABILIDADE_OMS=0)
STF= STF.assign(RESULTADO_OMS=0)
STF

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,NaN,0,0
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,NaN,0,0
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,NaN,0,0
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,NaN,0,0
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...
16518,2021-07-28,HC,203222,15347144135,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,NaN,0,0
16519,2021-08-02,HC,203222,15347162829,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,NaN,0,0
16520,2021-08-24,AR,2833,15347432581,stf,Ementa e Acórdão\n\n17/08/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,NaN,0,0
16521,2021-10-08,RE,1335917,15348181694,stf,Ementa e Acórdão\n\n27/09/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,NaN,0,0


#Criando coluna PROBABILIDADE E RESULTADO para fazer analise final

In [ ]:
#criando coluna "PROBABILIDADE"   e   "RESULTADO "
STF= STF.assign(PROBABILIDADE=0)
STF= STF.assign(RESULTADO=0)
STF

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,NaN,0,0,0,0
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,NaN,0,0,0,0
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,NaN,0,0,0,0
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,NaN,0,0,0,0
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16518,2021-07-28,HC,203222,15347144135,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,NaN,0,0,0,0
16519,2021-08-02,HC,203222,15347162829,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,NaN,0,0,0,0
16520,2021-08-24,AR,2833,15347432581,stf,Ementa e Acórdão\n\n17/08/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,NaN,0,0,0,0
16521,2021-10-08,RE,1335917,15348181694,stf,Ementa e Acórdão\n\n27/09/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,NaN,0,0,0,0


#Criando um DataFrame com os textos  :"OMS"

df=OMS

In [ ]:
# Dados que serão usados para criar o DataFrame
dados = {'Nome': [ 'oms', 'organizaçao mundial saude','world health organization','organização mundial da saúde - oms'],
         }

# Criar o DataFrame
OMS = pd.DataFrame(dados)

# Mostrar o DataFrame
OMS

,Nome
0,oms
1,organizaçao mundial saude
2,world health organization
3,organização mundial da saúde - oms


#Fazendo a comparaçao com o dataframe criado com a palavra OMS

In [ ]:
print (len(OMS['Nome'][0]))
print (len(OMS['Nome'][1]))
print (len(OMS['Nome'][2]))
print (len(OMS['Nome'][3]))

3
25
25
34


9 min

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
for i in range(len(STF)):
    texto_a_ser_comparado = STF['Conteudo'][i]
    P = []
    for j in range(len(OMS)):
        if len(OMS['Nome'][j]) ==3:
            texto_fonte = OMS['Nome'][j]  # Ajuste aqui para acessar a coluna 'Conteudo' em OMS
            n = 3
            counts = CountVectorizer(analyzer='word', ngram_range=(n, n))
            vocab2int = counts.fit([texto_a_ser_comparado, texto_fonte]).vocabulary_
            n_grams = counts.fit_transform([texto_a_ser_comparado, texto_fonte])
            n_grams_array = n_grams.toarray()
            intersection_list = np.amin(n_grams.toarray(), axis=0)
            intersection_count = np.sum(intersection_list)
            index_A = 0
            A_count = np.sum(n_grams.toarray()[index_A])
            p = intersection_count / A_count
            pt = round(p, 2)
            P.append(pt)
        elif len(OMS['Nome'][j]) ==25:
            texto_fonte = OMS['Nome'][j]  # Ajuste aqui para acessar a coluna 'Conteudo' em OMS
            n = 1
            counts = CountVectorizer(analyzer='word', ngram_range=(n, n))
            vocab2int = counts.fit([texto_a_ser_comparado, texto_fonte]).vocabulary_
            n_grams = counts.fit_transform([texto_a_ser_comparado, texto_fonte])
            n_grams_array = n_grams.toarray()
            intersection_list = np.amin(n_grams.toarray(), axis=0)
            intersection_count = np.sum(intersection_list)
            index_A = 0
            A_count = np.sum(n_grams.toarray()[index_A])
            p = intersection_count / A_count
            pt = round(p, 2)
            P.append(pt)
        elif len(OMS['Nome'][j]) ==34:
            texto_fonte = OMS['Nome'][j]  # Ajuste aqui para acessar a coluna 'Conteudo' em OMS
            n = 1
            counts = CountVectorizer(analyzer='word', ngram_range=(n, n))
            vocab2int = counts.fit([texto_a_ser_comparado, texto_fonte]).vocabulary_
            n_grams = counts.fit_transform([texto_a_ser_comparado, texto_fonte])
            n_grams_array = n_grams.toarray()
            intersection_list = np.amin(n_grams.toarray(), axis=0)
            intersection_count = np.sum(intersection_list)
            index_A = 0
            A_count = np.sum(n_grams.toarray()[index_A])
            p = intersection_count / A_count
            pt = round(p, 2)
            P.append(pt)
    probabilidade_maxima = max(P)
    localizacao = P.index(probabilidade_maxima)
    STF.loc[i, 'PROBABILIDADE_OMS'] = probabilidade_maxima
    STF.loc[i, 'RESULTADO_OMS'] = localizacao

<ipython-input-10-938f1ea5f504>:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  STF.loc[i, 'PROBABILIDADE_OMS'] = probabilidade_maxima


#separando stf nao e sim

In [ ]:
for i in range(len(STF)):
  if STF['PROBABILIDADE_OMS'][i]!=0:
    STF['OMS'][i]='sim'
  else:
    STF['OMS'][i]='não'

<ipython-input-11-e4c66469d9f8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  STF['OMS'][i]='não'
<ipython-input-11-e4c66469d9f8>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'não' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  STF['OMS'][i]='não'


In [ ]:
## Filtrando o DataFrame com base em um valor específico da coluna 'Grupo'
grupo_especifico = 'sim'
GRUPO_SIM = STF[STF['OMS'] =='sim']
GRUPO_SIM_DESORDEM=GRUPO_SIM
GRUPO_SIM_DESORDEM

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
5,2004-08-31,RE,199272,1627423,stf,DECISÃO: Atendo ao agravo e reconsidero a deci...,COVID-19,sim,0.01,3,0,0
34,2011-11-16,Rcl,2780,1605851,stf,DJe 05/12/2011\n\nEmenta e Acórdão\n\n16/11/20...,COVID-19,sim,0.01,3,0,0
42,2012-09-28,RE,701022,101824085,stf,RECURSO EXTRAORDINÁRIO 701.022 SANTA CATARINA\...,COVID-19,sim,0.01,3,0,0
43,2012-10-03,RE,701022,101824085,stf,RECURSO EXTRAORDINÁRIO 701.022 SANTA CATARINA\...,COVID-19,sim,0.01,3,0,0
44,2012-10-10,RE,542485,101445730,stf,AG.REG. NO RECURSO EXTRAORDINÁRIO 542.485 RIO ...,COVID-19,sim,0.01,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16470,2008-05-29,AI,715019,2811652,stf,Trata-se de agravo de instrumento contra decis...,Zika vírus_Zika virus disease,sim,0.01,3,0,0
16471,2008-06-09,AI,715019,2378326,stf,Trata-se de agravo de instrumento contra decis...,Zika vírus_Zika virus disease,sim,0.01,3,0,0
16472,2009-03-09,RE,596025,3093042,stf,DECISÃO: 1. Trata-se de recurso extraordinário...,Zika vírus_Zika virus disease,sim,0.01,3,0,0
16473,2009-03-17,RE,596025,2564402,stf,DECISÃO: 1. Trata-se de recurso extraordinário...,Zika vírus_Zika virus disease,sim,0.01,3,0,0


In [ ]:
len(GRUPO_SIM)

1517

In [ ]:
# Filtrando ccom aqueles que não mencionam oms

GRUPO_NAO = STF[STF['OMS'] == 'não']
GRUPO_NAO

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,não,0.0,0,0,0
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,não,0.0,0,0,0
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,não,0.0,0,0,0
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,não,0.0,0,0,0
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,não,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16518,2021-07-28,HC,203222,15347144135,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,não,0.0,0,0,0
16519,2021-08-02,HC,203222,15347162829,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,não,0.0,0,0,0
16520,2021-08-24,AR,2833,15347432581,stf,Ementa e Acórdão\n\n17/08/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,não,0.0,0,0,0
16521,2021-10-08,RE,1335917,15348181694,stf,Ementa e Acórdão\n\n27/09/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,não,0.0,0,0,0


In [ ]:
len(GRUPO_NAO)

15006

ordenei o grupo sim

In [ ]:
GRUPO_SIM


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
5,2004-08-31,RE,199272,1627423,stf,DECISÃO: Atendo ao agravo e reconsidero a deci...,COVID-19,sim,0.01,3,0,0
34,2011-11-16,Rcl,2780,1605851,stf,DJe 05/12/2011\n\nEmenta e Acórdão\n\n16/11/20...,COVID-19,sim,0.01,3,0,0
42,2012-09-28,RE,701022,101824085,stf,RECURSO EXTRAORDINÁRIO 701.022 SANTA CATARINA\...,COVID-19,sim,0.01,3,0,0
43,2012-10-03,RE,701022,101824085,stf,RECURSO EXTRAORDINÁRIO 701.022 SANTA CATARINA\...,COVID-19,sim,0.01,3,0,0
44,2012-10-10,RE,542485,101445730,stf,AG.REG. NO RECURSO EXTRAORDINÁRIO 542.485 RIO ...,COVID-19,sim,0.01,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16470,2008-05-29,AI,715019,2811652,stf,Trata-se de agravo de instrumento contra decis...,Zika vírus_Zika virus disease,sim,0.01,3,0,0
16471,2008-06-09,AI,715019,2378326,stf,Trata-se de agravo de instrumento contra decis...,Zika vírus_Zika virus disease,sim,0.01,3,0,0
16472,2009-03-09,RE,596025,3093042,stf,DECISÃO: 1. Trata-se de recurso extraordinário...,Zika vírus_Zika virus disease,sim,0.01,3,0,0
16473,2009-03-17,RE,596025,2564402,stf,DECISÃO: 1. Trata-se de recurso extraordinário...,Zika vírus_Zika virus disease,sim,0.01,3,0,0


In [ ]:
GRUPO_SIM.reset_index(drop=True, inplace=True)
GRUPO_SIM_ORDENADO=GRUPO_SIM
GRUPO_SIM_ORDENADO

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
0,2004-08-31,RE,199272,1627423,stf,DECISÃO: Atendo ao agravo e reconsidero a deci...,COVID-19,sim,0.01,3,0,0
1,2011-11-16,Rcl,2780,1605851,stf,DJe 05/12/2011\n\nEmenta e Acórdão\n\n16/11/20...,COVID-19,sim,0.01,3,0,0
2,2012-09-28,RE,701022,101824085,stf,RECURSO EXTRAORDINÁRIO 701.022 SANTA CATARINA\...,COVID-19,sim,0.01,3,0,0
3,2012-10-03,RE,701022,101824085,stf,RECURSO EXTRAORDINÁRIO 701.022 SANTA CATARINA\...,COVID-19,sim,0.01,3,0,0
4,2012-10-10,RE,542485,101445730,stf,AG.REG. NO RECURSO EXTRAORDINÁRIO 542.485 RIO ...,COVID-19,sim,0.01,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1512,2008-05-29,AI,715019,2811652,stf,Trata-se de agravo de instrumento contra decis...,Zika vírus_Zika virus disease,sim,0.01,3,0,0
1513,2008-06-09,AI,715019,2378326,stf,Trata-se de agravo de instrumento contra decis...,Zika vírus_Zika virus disease,sim,0.01,3,0,0
1514,2009-03-09,RE,596025,3093042,stf,DECISÃO: 1. Trata-se de recurso extraordinário...,Zika vírus_Zika virus disease,sim,0.01,3,0,0
1515,2009-03-17,RE,596025,2564402,stf,DECISÃO: 1. Trata-se de recurso extraordinário...,Zika vírus_Zika virus disease,sim,0.01,3,0,0


In [ ]:
GRUPO_NAO.reset_index(drop=True, inplace=True)
GRUPO_NAO

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,não,0.0,0,0,0
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,não,0.0,0,0,0
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,não,0.0,0,0,0
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,não,0.0,0,0,0
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,não,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15001,2021-07-28,HC,203222,15347144135,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,não,0.0,0,0,0
15002,2021-08-02,HC,203222,15347162829,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,não,0.0,0,0,0
15003,2021-08-24,AR,2833,15347432581,stf,Ementa e Acórdão\n\n17/08/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,não,0.0,0,0,0
15004,2021-10-08,RE,1335917,15348181694,stf,Ementa e Acórdão\n\n27/09/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,não,0.0,0,0,0


In [ ]:
import pandas as pd


# Definir os tamanhos dos grupos
tamanhos = [500]*30 + [6]

# Dividir o DataFrame em grupos com os tamanhos especificados
grupos = [GRUPO_NAO.iloc[sum(tamanhos[:i]):sum(tamanhos[:i + 1])] for i in range(len(tamanhos))]

# Mostrar o grupo 4
grupo_1= grupos[0]
grupo_2= grupos[1]
grupo_3= grupos[2]
grupo_4 = grupos[3]
grupo_5 = grupos[4]
grupo_6 = grupos[5]

grupo_7= grupos[6]
grupo_8= grupos[7]
grupo_9= grupos[8]
grupo_10 = grupos[9]
grupo_11 = grupos[10]
grupo_12 = grupos[11]
grupo_13 =grupos[12]
grupo_14=grupos[13]
grupo_15=grupos[14]
grupo_16=grupos[15]

grupo_17= grupos[16]
grupo_18= grupos[17]
grupo_19= grupos[18]
grupo_20 = grupos[19]
grupo_21 = grupos[20]
grupo_22 = grupos[21]

grupo_23= grupos[22]
grupo_24= grupos[23]
grupo_25= grupos[24]
grupo_26 = grupos[25]
grupo_27 = grupos[26]
grupo_28 = grupos[27]
grupo_29 =grupos[28]
grupo_30=grupos[29]
grupo_31=grupos[30]

#grupo 5

In [ ]:
grupo_5

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
2000,2021-01-25,Rcl,45419,15345471369,stf,RECLAMAÇÃO 45.419 ESPÍRITO SANTO\n\nRELATOR : ...,COVID-19,não,0.0,0,0,0
2001,2021-01-25,RHC,195648,15345466033,stf,RECURSO ORDINÁRIO EM HABEAS CORPUS 195.648 PAR...,COVID-19,não,0.0,0,0,0
2002,2021-01-25,RHC,195980,15345466030,stf,RECURSO ORDINÁRIO EM HABEAS CORPUS 195.980 PAR...,COVID-19,não,0.0,0,0,0
2003,2021-01-26,ADPF,754,15345473470,stf,ARGUIÇÃO DE DESCUMPRIMENTO DE PRECEITO FUNDAME...,COVID-19,não,0.0,0,0,0
2004,2021-01-26,HC,195217,15345474303,stf,HABEAS CORPUS 195.217 SÃO PAULO\n\nRELATORA : ...,COVID-19,não,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2021-02-22,HC,197837,15345690298,stf,HABEAS CORPUS 197.837 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0,0
2496,2021-02-22,HC,197842,15345700500,stf,HABEAS CORPUS 197.842 SÃO PAULO\n\nREGISTRADO ...,COVID-19,não,0.0,0,0,0
2497,2021-02-22,HC,197856,15345698830,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 197.856 SÃO P...,COVID-19,não,0.0,0,0,0
2498,2021-02-22,PPE,968,15345680259,stf,PRISÃO PREVENTIVA PARA EXTRADIÇÃO 968 DISTRITO...,COVID-19,não,0.0,0,0,0


In [ ]:
grupo_5.reset_index(drop=True, inplace=True)
grupo_5

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
0,2021-01-25,Rcl,45419,15345471369,stf,RECLAMAÇÃO 45.419 ESPÍRITO SANTO\n\nRELATOR : ...,COVID-19,não,0.0,0,0,0
1,2021-01-25,RHC,195648,15345466033,stf,RECURSO ORDINÁRIO EM HABEAS CORPUS 195.648 PAR...,COVID-19,não,0.0,0,0,0
2,2021-01-25,RHC,195980,15345466030,stf,RECURSO ORDINÁRIO EM HABEAS CORPUS 195.980 PAR...,COVID-19,não,0.0,0,0,0
3,2021-01-26,ADPF,754,15345473470,stf,ARGUIÇÃO DE DESCUMPRIMENTO DE PRECEITO FUNDAME...,COVID-19,não,0.0,0,0,0
4,2021-01-26,HC,195217,15345474303,stf,HABEAS CORPUS 195.217 SÃO PAULO\n\nRELATORA : ...,COVID-19,não,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,2021-02-22,HC,197837,15345690298,stf,HABEAS CORPUS 197.837 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0,0
496,2021-02-22,HC,197842,15345700500,stf,HABEAS CORPUS 197.842 SÃO PAULO\n\nREGISTRADO ...,COVID-19,não,0.0,0,0,0
497,2021-02-22,HC,197856,15345698830,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 197.856 SÃO P...,COVID-19,não,0.0,0,0,0
498,2021-02-22,PPE,968,15345680259,stf,PRISÃO PREVENTIVA PARA EXTRADIÇÃO 968 DISTRITO...,COVID-19,não,0.0,0,0,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
for i in range(len(grupo_5)):
  texto_a_ser_comparado =grupo_5["Conteudo"][i]
  #texto_a_ser_comparado =GRUPO_NAO.iloc[i]['Conteudo']
  P=[]
  #for j in range(len(df1111)):
  for j in range(len(GRUPO_SIM)):
    #valor = GRUPO_SIM.iloc[0]['Conteudo']
    #texto_fonte=GRUPO_SIM.iloc[j]['Conteudo']
    texto_fonte= GRUPO_SIM["Conteudo"][j]
    n=1
    #instancia o contador de n-grams
    counts=CountVectorizer(analyzer='word',ngram_range=(n,n))
    #cria um dicionario de n grams
    vocab2int=counts.fit([texto_a_ser_comparado,texto_fonte]).vocabulary_
    n_grams=counts.fit_transform([texto_a_ser_comparado,texto_fonte])
    #cria uma matriz de contagem para os dois textos
    n_grams_array = n_grams.toarray()
    #matriz de

    intersection_list= np.amin(n_grams.toarray(),axis=0)
    #intersection_list= np.amin(n_grams.toarray())
    intersection_list
   #contagem interseçao
    intersection_count= np.sum(intersection_list)
    index_A=0
    A_count=np.sum(n_grams.toarray()[index_A])
    p=intersection_count/A_count
    pt = round(p, 2)
    P.append(pt)
    #colocar o vetor e adicionar este p no vetor
  B=[]
  Z=[]
  probabilidade_maxima = max(P)
  localizaçao  = P.index(probabilidade_maxima)
  grupo_5['PROBABILIDADE'][i]=probabilidade_maxima
  grupo_5['RESULTADO'][i]=localizaçao
  #df2222['PROBABILIDADE'][i]=probabilidade_maxima
  #df2222['RESULTADO_df1'][i]=localizaçao

A saída de streaming foi truncada nas últimas 5000 linhas.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_5['PROBABILIDADE'][i]=probabilidade_maxima
<ipython-input-21-d8629b4cf0a8>:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.4' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  grupo_5['PROBABILIDADE'][i]=probabilidade_maxima
<ipython-input-21-d8629b4cf0a8>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_5['RESULTADO'][i]=localizaçao
<ipython-input-21-d8629b4cf0a8>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in t

In [ ]:
GRUPO_SIM['Data'] = pd.to_datetime(GRUPO_SIM['Data'])
grupo_5['Data'] = pd.to_datetime(grupo_5['Data'])

<ipython-input-24-104a7dac02c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_SIM['Data'] = pd.to_datetime(GRUPO_SIM['Data'])
<ipython-input-24-104a7dac02c1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_5['Data'] = pd.to_datetime(grupo_5['Data'])


In [ ]:
condicao = grupo_5['PROBABILIDADE'] > 0.7

# Filtrar o DataFrame com base na condição
grupo_5 = grupo_5[condicao]

# Exibir o DataFrame filtrado
grupo_5

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
121,2021-02-02,HC,195933,15345525738,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 195.933 SÃO P...,COVID-19,não,0.0,0,0.75,357
145,2021-02-03,HC,196252,15345526237,stf,HABEAS CORPUS 196.252 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.73,38
146,2021-02-03,HC,196272,15345526181,stf,HABEAS CORPUS 196.272 SANTA CATARINA\n\nRELATO...,COVID-19,não,0.0,0,0.72,38
150,2021-02-03,HC,196484,15345526152,stf,HABEAS CORPUS 196.484 DISTRITO FEDERAL\n\nRELA...,COVID-19,não,0.0,0,0.83,38
151,2021-02-03,HC,196499,15345526394,stf,HABEAS CORPUS 196.499 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0.79,38
152,2021-02-03,HC,196541,15345526481,stf,HABEAS CORPUS 196.541 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0.73,38
154,2021-02-03,HC,196681,15345526367,stf,HABEAS CORPUS 196.681 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.71,389
155,2021-02-03,HC,196699,15345526375,stf,HABEAS CORPUS 196.699 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0.78,38
157,2021-02-03,HC,196710,15345526385,stf,HABEAS CORPUS 196.710 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0.74,389
160,2021-02-03,HC,197117,15345526743,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 197.117 GOIÁS...,COVID-19,não,0.0,0,0.73,148


In [ ]:
#criando uma coluna repetindo o indice neste dataframe GRUPO_SIM
GRUPO_SIM['indice'] = GRUPO_SIM.index
GRUPO_SIM

<ipython-input-26-f7d753fe9fe5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_SIM['indice'] = GRUPO_SIM.index


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,indice
0,2004-08-31,RE,199272,1627423,stf,DECISÃO: Atendo ao agravo e reconsidero a deci...,COVID-19,sim,0.01,3,0,0,0
1,2011-11-16,Rcl,2780,1605851,stf,DJe 05/12/2011\n\nEmenta e Acórdão\n\n16/11/20...,COVID-19,sim,0.01,3,0,0,1
2,2012-09-28,RE,701022,101824085,stf,RECURSO EXTRAORDINÁRIO 701.022 SANTA CATARINA\...,COVID-19,sim,0.01,3,0,0,2
3,2012-10-03,RE,701022,101824085,stf,RECURSO EXTRAORDINÁRIO 701.022 SANTA CATARINA\...,COVID-19,sim,0.01,3,0,0,3
4,2012-10-10,RE,542485,101445730,stf,AG.REG. NO RECURSO EXTRAORDINÁRIO 542.485 RIO ...,COVID-19,sim,0.01,3,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512,2008-05-29,AI,715019,2811652,stf,Trata-se de agravo de instrumento contra decis...,Zika vírus_Zika virus disease,sim,0.01,3,0,0,1512
1513,2008-06-09,AI,715019,2378326,stf,Trata-se de agravo de instrumento contra decis...,Zika vírus_Zika virus disease,sim,0.01,3,0,0,1513
1514,2009-03-09,RE,596025,3093042,stf,DECISÃO: 1. Trata-se de recurso extraordinário...,Zika vírus_Zika virus disease,sim,0.01,3,0,0,1514
1515,2009-03-17,RE,596025,2564402,stf,DECISÃO: 1. Trata-se de recurso extraordinário...,Zika vírus_Zika virus disease,sim,0.01,3,0,0,1515


In [ ]:
# Usando o método map para colocar no dataframe (grupo_1) a coluna com as datas relacionadas ao indice do GRUPO_SIM
grupo_5['Data_SIM'] =grupo_5['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Data'])
grupo_5

<ipython-input-27-784e6bc1c6d4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_5['Data_SIM'] =grupo_5['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Data'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM
121,2021-02-02,HC,195933,15345525738,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 195.933 SÃO P...,COVID-19,não,0.0,0,0.75,357,2021-04-06
145,2021-02-03,HC,196252,15345526237,stf,HABEAS CORPUS 196.252 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.73,38,2020-04-02
146,2021-02-03,HC,196272,15345526181,stf,HABEAS CORPUS 196.272 SANTA CATARINA\n\nRELATO...,COVID-19,não,0.0,0,0.72,38,2020-04-02
150,2021-02-03,HC,196484,15345526152,stf,HABEAS CORPUS 196.484 DISTRITO FEDERAL\n\nRELA...,COVID-19,não,0.0,0,0.83,38,2020-04-02
151,2021-02-03,HC,196499,15345526394,stf,HABEAS CORPUS 196.499 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0.79,38,2020-04-02
152,2021-02-03,HC,196541,15345526481,stf,HABEAS CORPUS 196.541 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0.73,38,2020-04-02
154,2021-02-03,HC,196681,15345526367,stf,HABEAS CORPUS 196.681 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.71,389,2021-04-23
155,2021-02-03,HC,196699,15345526375,stf,HABEAS CORPUS 196.699 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0.78,38,2020-04-02
157,2021-02-03,HC,196710,15345526385,stf,HABEAS CORPUS 196.710 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0.74,389,2021-04-23
160,2021-02-03,HC,197117,15345526743,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 197.117 GOIÁS...,COVID-19,não,0.0,0,0.73,148,2020-11-05


In [ ]:
# Usando o método map para colocar no dataframe (grupo_1) a coluna com as datas relacionadas ao indice do GRUPO_SIM
grupo_5['Número_SIM'] =grupo_5['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Número'])
grupo_5['Doc ID_SIM'] =grupo_5['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Doc ID'])
grupo_5

<ipython-input-28-16a5fac962e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_5['Número_SIM'] =grupo_5['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Número'])
<ipython-input-28-16a5fac962e8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_5['Doc ID_SIM'] =grupo_5['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Doc ID'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM,Número_SIM,Doc ID_SIM
121,2021-02-02,HC,195933,15345525738,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 195.933 SÃO P...,COVID-19,não,0.0,0,0.75,357,2021-04-06,199683,15346080062
145,2021-02-03,HC,196252,15345526237,stf,HABEAS CORPUS 196.252 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.73,38,2020-04-02,183275,15342797616
146,2021-02-03,HC,196272,15345526181,stf,HABEAS CORPUS 196.272 SANTA CATARINA\n\nRELATO...,COVID-19,não,0.0,0,0.72,38,2020-04-02,183275,15342797616
150,2021-02-03,HC,196484,15345526152,stf,HABEAS CORPUS 196.484 DISTRITO FEDERAL\n\nRELA...,COVID-19,não,0.0,0,0.83,38,2020-04-02,183275,15342797616
151,2021-02-03,HC,196499,15345526394,stf,HABEAS CORPUS 196.499 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0.79,38,2020-04-02,183275,15342797616
152,2021-02-03,HC,196541,15345526481,stf,HABEAS CORPUS 196.541 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0.73,38,2020-04-02,183275,15342797616
154,2021-02-03,HC,196681,15345526367,stf,HABEAS CORPUS 196.681 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.71,389,2021-04-23,200231,15346220661
155,2021-02-03,HC,196699,15345526375,stf,HABEAS CORPUS 196.699 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0.78,38,2020-04-02,183275,15342797616
157,2021-02-03,HC,196710,15345526385,stf,HABEAS CORPUS 196.710 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0.74,389,2021-04-23,200231,15346220661
160,2021-02-03,HC,197117,15345526743,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 197.117 GOIÁS...,COVID-19,não,0.0,0,0.73,148,2020-11-05,192869,15344847969


In [ ]:
#grupo_22['Data'] = pd.to_datetime(grupo_22['Data'])
#grupo_22['Data_SIM'] = pd.to_datetime(grupo_22['Data_SIM'])
#grupo_22 = grupo_22[grupo_22['Data'] > grupo_22['Data_SIM']]
#grupo_22




In [ ]:
# Usando o método map para colocar no dataframe (grupo_1) a coluna com as datas relacionadas ao indice do GRUPO_SIM
grupo_5['tipo_SIM'] =grupo_5['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Tipo'])
grupo_5

<ipython-input-29-726036440430>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_5['tipo_SIM'] =grupo_5['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Tipo'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM,Número_SIM,Doc ID_SIM,tipo_SIM
121,2021-02-02,HC,195933,15345525738,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 195.933 SÃO P...,COVID-19,não,0.0,0,0.75,357,2021-04-06,199683,15346080062,HC
145,2021-02-03,HC,196252,15345526237,stf,HABEAS CORPUS 196.252 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.73,38,2020-04-02,183275,15342797616,HC
146,2021-02-03,HC,196272,15345526181,stf,HABEAS CORPUS 196.272 SANTA CATARINA\n\nRELATO...,COVID-19,não,0.0,0,0.72,38,2020-04-02,183275,15342797616,HC
150,2021-02-03,HC,196484,15345526152,stf,HABEAS CORPUS 196.484 DISTRITO FEDERAL\n\nRELA...,COVID-19,não,0.0,0,0.83,38,2020-04-02,183275,15342797616,HC
151,2021-02-03,HC,196499,15345526394,stf,HABEAS CORPUS 196.499 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0.79,38,2020-04-02,183275,15342797616,HC
152,2021-02-03,HC,196541,15345526481,stf,HABEAS CORPUS 196.541 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0.73,38,2020-04-02,183275,15342797616,HC
154,2021-02-03,HC,196681,15345526367,stf,HABEAS CORPUS 196.681 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.71,389,2021-04-23,200231,15346220661,HC
155,2021-02-03,HC,196699,15345526375,stf,HABEAS CORPUS 196.699 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0.78,38,2020-04-02,183275,15342797616,HC
157,2021-02-03,HC,196710,15345526385,stf,HABEAS CORPUS 196.710 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0.74,389,2021-04-23,200231,15346220661,HC
160,2021-02-03,HC,197117,15345526743,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 197.117 GOIÁS...,COVID-19,não,0.0,0,0.73,148,2020-11-05,192869,15344847969,HC


In [ ]:
nome_do_arquivo = 'grupo_5.xlsx'
grupo_5.to_excel(nome_do_arquivo, index=False)

# Local onde o arquivo é salvo no Google Colab
print("O arquivo foi salvo em:", nome_do_arquivo)

O arquivo foi salvo em: grupo_5.xlsx


In [ ]:
from google.colab import drive
import shutil

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o caminho para o arquivo Excel no sistema de arquivos temporário do Google Colab
caminho_arquivo_colab = 'grupo_5.xlsx'

# Definir o caminho para onde queremos mover o arquivo no Google Drive
caminho_destino_drive = '/content/drive/MyDrive/grupo_5.xlsx'

# Mover o arquivo para o Google Drive
shutil.move(caminho_arquivo_colab, caminho_destino_drive)

print("Arquivo movido com sucesso para o Google Drive em:", caminho_destino_drive)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Arquivo movido com sucesso para o Google Drive em: /content/drive/MyDrive/grupo_5.xlsx


grupo 6

In [ ]:
grupo_6.reset_index(drop=True, inplace=True)
grupo_6

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
0,2021-02-22,Rcl,45329,15345712479,stf,EXTENSÃO NA RECLAMAÇÃO 45.329 SÃO PAULO\n\nREL...,COVID-19,não,0.0,0,0,0
1,2021-02-22,Rcl,45461,15345691056,stf,RECLAMAÇÃO 45.461 SANTA CATARINA\n\nRELATOR : ...,COVID-19,não,0.0,0,0,0
2,2021-02-22,Rcl,45618,15345691041,stf,RECLAMAÇÃO 45.618 SÃO PAULO\n\nRELATOR : MIN. ...,COVID-19,não,0.0,0,0,0
3,2021-02-22,Rcl,45715,15345712760,stf,RECLAMAÇÃO 45.715 SÃO PAULO\n\nRELATORA : MIN....,COVID-19,não,0.0,0,0,0
4,2021-02-22,Rcl,45759,15345691411,stf,RECLAMAÇÃO 45.759 SÃO PAULO\n\nRELATORA : MIN....,COVID-19,não,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,2021-03-18,HC,198781,15345936243,stf,HABEAS CORPUS 198.781 SERGIPE\n\nRELATORA : MI...,COVID-19,não,0.0,0,0,0
496,2021-03-18,HC,198881,15345935867,stf,HABEAS CORPUS 198.881 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0,0
497,2021-03-18,HC,198923,15345936740,stf,HABEAS CORPUS 198.923 SÃO PAULO\n\nRELATORA : ...,COVID-19,não,0.0,0,0,0
498,2021-03-18,HC,198942,15345935599,stf,HABEAS CORPUS 198.942 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
for i in range(len(grupo_6)):
  texto_a_ser_comparado =grupo_6["Conteudo"][i]
  #texto_a_ser_comparado =GRUPO_NAO.iloc[i]['Conteudo']
  P=[]
  #for j in range(len(df1111)):
  for j in range(len(GRUPO_SIM)):
    #valor = GRUPO_SIM.iloc[0]['Conteudo']
    #texto_fonte=GRUPO_SIM.iloc[j]['Conteudo']
    texto_fonte= GRUPO_SIM["Conteudo"][j]
    n=1
    #instancia o contador de n-grams
    counts=CountVectorizer(analyzer='word',ngram_range=(n,n))
    #cria um dicionario de n grams
    vocab2int=counts.fit([texto_a_ser_comparado,texto_fonte]).vocabulary_
    n_grams=counts.fit_transform([texto_a_ser_comparado,texto_fonte])
    #cria uma matriz de contagem para os dois textos
    n_grams_array = n_grams.toarray()
    #matriz de

    intersection_list= np.amin(n_grams.toarray(),axis=0)
    #intersection_list= np.amin(n_grams.toarray())
    intersection_list
   #contagem interseçao
    intersection_count= np.sum(intersection_list)
    index_A=0
    A_count=np.sum(n_grams.toarray()[index_A])
    p=intersection_count/A_count
    pt = round(p, 2)
    P.append(pt)
    #colocar o vetor e adicionar este p no vetor
  B=[]
  Z=[]
  probabilidade_maxima = max(P)
  localizaçao  = P.index(probabilidade_maxima)
  grupo_6['PROBABILIDADE'][i]=probabilidade_maxima
  grupo_6['RESULTADO'][i]=localizaçao
  #df2222['PROBABILIDADE'][i]=probabilidade_maxima
  #df2222['RESULTADO_df1'][i]=localizaçao

A saída de streaming foi truncada nas últimas 5000 linhas.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_6['PROBABILIDADE'][i]=probabilidade_maxima
<ipython-input-34-9256648e804c>:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.29' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  grupo_6['PROBABILIDADE'][i]=probabilidade_maxima
<ipython-input-34-9256648e804c>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_6['RESULTADO'][i]=localizaçao
<ipython-input-34-9256648e804c>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

In [ ]:
GRUPO_SIM['Data'] = pd.to_datetime(GRUPO_SIM['Data'])
grupo_6['Data'] = pd.to_datetime(grupo_6['Data'])

<ipython-input-35-4d54dc7bae6e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_SIM['Data'] = pd.to_datetime(GRUPO_SIM['Data'])
<ipython-input-35-4d54dc7bae6e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_6['Data'] = pd.to_datetime(grupo_6['Data'])


In [ ]:
condicao = grupo_6['PROBABILIDADE'] > 0.7

# Filtrar o DataFrame com base na condição
grupo_6 = grupo_6[condicao]

# Exibir o DataFrame filtrado
grupo_6

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
22,2021-02-23,MS,37647,15345716782,stf,me MP n° 2.200-2/2001 de 24/08/2001. O documen...,COVID-19,não,0.0,0,0.84,293
44,2021-02-24,HC,197921,15345716744,stf,HABEAS CORPUS 197.921 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.76,38
195,2021-03-04,HC,198212,15345803466,stf,HABEAS CORPUS 198.212 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.71,38
310,2021-03-10,HC,198482,15345858301,stf,HABEAS CORPUS 198.482 GOIÁS\n\nRELATOR : MIN. ...,COVID-19,não,0.0,0,0.73,38
395,2021-03-15,HC,198517,15345907121,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 198.517 MINAS...,COVID-19,não,0.0,0,0.72,71
396,2021-03-15,HC,198574,15345906396,stf,HABEAS CORPUS 198.574 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.73,38
453,2021-03-17,HC,196383,15345928785,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 196.383 SÃO P...,COVID-19,não,0.0,0,0.77,449


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
0,2002-03-26,RE,207132,240928,stf,"256 \n\n&t'9""?'cmo. ~tuna/ cfJ}cde?'al \n\nCOO...",COVID-19,não,0.0,0,0,0
1,2002-05-29,RE,134509,207766,stf,Sepromo Tribunal Federal 364\nCOORD. DE ANÁLIS...,COVID-19,não,0.0,0,0,0
2,2003-05-14,AI,418302,311508,stf,AI-AgR 418302Ementa e Acórdão (1)\n\n1184 \n\n...,COVID-19,não,0.0,0,0,0
3,2003-10-08,Rcl,1935,87257,stf,341 \n\nTRIBUNAL PLENO \n\nCOORD. DE ANÁLISE D...,COVID-19,não,0.0,0,0,0
4,2003-10-23,ACO,529,1263457,stf,DESPACHO :\n\n1. SÍNTESE DOS FATOS.\n\nTrata-s...,COVID-19,não,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16518,2021-07-28,HC,203222,15347144135,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,não,0.0,0,0,0
16519,2021-08-02,HC,203222,15347162829,stf,Documento assinado eletronicamente pelo(a) Min...,Zika vírus_Zika virus disease,não,0.0,0,0,0
16520,2021-08-24,AR,2833,15347432581,stf,Ementa e Acórdão\n\n17/08/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,não,0.0,0,0,0
16521,2021-10-08,RE,1335917,15348181694,stf,Ementa e Acórdão\n\n27/09/2021 PLENÁRIO\n\nAG....,Zika vírus_Zika virus disease,não,0.0,0,0,0


In [ ]:
#criando uma coluna repetindo o indice neste dataframe GRUPO_SIM
GRUPO_SIM['indice'] = GRUPO_SIM.index
GRUPO_SIM

<ipython-input-37-f7d753fe9fe5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_SIM['indice'] = GRUPO_SIM.index


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,indice
0,2004-08-31,RE,199272,1627423,stf,DECISÃO: Atendo ao agravo e reconsidero a deci...,COVID-19,sim,0.01,3,0,0,0
1,2011-11-16,Rcl,2780,1605851,stf,DJe 05/12/2011\n\nEmenta e Acórdão\n\n16/11/20...,COVID-19,sim,0.01,3,0,0,1
2,2012-09-28,RE,701022,101824085,stf,RECURSO EXTRAORDINÁRIO 701.022 SANTA CATARINA\...,COVID-19,sim,0.01,3,0,0,2
3,2012-10-03,RE,701022,101824085,stf,RECURSO EXTRAORDINÁRIO 701.022 SANTA CATARINA\...,COVID-19,sim,0.01,3,0,0,3
4,2012-10-10,RE,542485,101445730,stf,AG.REG. NO RECURSO EXTRAORDINÁRIO 542.485 RIO ...,COVID-19,sim,0.01,3,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512,2008-05-29,AI,715019,2811652,stf,Trata-se de agravo de instrumento contra decis...,Zika vírus_Zika virus disease,sim,0.01,3,0,0,1512
1513,2008-06-09,AI,715019,2378326,stf,Trata-se de agravo de instrumento contra decis...,Zika vírus_Zika virus disease,sim,0.01,3,0,0,1513
1514,2009-03-09,RE,596025,3093042,stf,DECISÃO: 1. Trata-se de recurso extraordinário...,Zika vírus_Zika virus disease,sim,0.01,3,0,0,1514
1515,2009-03-17,RE,596025,2564402,stf,DECISÃO: 1. Trata-se de recurso extraordinário...,Zika vírus_Zika virus disease,sim,0.01,3,0,0,1515


In [ ]:
# Usando o método map para colocar no dataframe (grupo_1) a coluna com as datas relacionadas ao indice do GRUPO_SIM
grupo_6['Data_SIM'] =grupo_6['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Data'])
grupo_6

<ipython-input-38-ededbe8f822d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_6['Data_SIM'] =grupo_6['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Data'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM
22,2021-02-23,MS,37647,15345716782,stf,me MP n° 2.200-2/2001 de 24/08/2001. O documen...,COVID-19,não,0.0,0,0.84,293,2021-02-24
44,2021-02-24,HC,197921,15345716744,stf,HABEAS CORPUS 197.921 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.76,38,2020-04-02
195,2021-03-04,HC,198212,15345803466,stf,HABEAS CORPUS 198.212 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.71,38,2020-04-02
310,2021-03-10,HC,198482,15345858301,stf,HABEAS CORPUS 198.482 GOIÁS\n\nRELATOR : MIN. ...,COVID-19,não,0.0,0,0.73,38,2020-04-02
395,2021-03-15,HC,198517,15345907121,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 198.517 MINAS...,COVID-19,não,0.0,0,0.72,71,2020-04-16
396,2021-03-15,HC,198574,15345906396,stf,HABEAS CORPUS 198.574 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.73,38,2020-04-02
453,2021-03-17,HC,196383,15345928785,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 196.383 SÃO P...,COVID-19,não,0.0,0,0.77,449,2021-05-28


In [ ]:
# Usando o método map para colocar no dataframe (grupo_1) a coluna com as datas relacionadas ao indice do GRUPO_SIM
grupo_6['Número_SIM'] =grupo_6['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Número'])
grupo_6['Doc ID_SIM'] =grupo_6['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Doc ID'])
grupo_6

<ipython-input-39-717c65fb9bc4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_6['Número_SIM'] =grupo_6['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Número'])
<ipython-input-39-717c65fb9bc4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_6['Doc ID_SIM'] =grupo_6['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Doc ID'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM,Número_SIM,Doc ID_SIM
22,2021-02-23,MS,37647,15345716782,stf,me MP n° 2.200-2/2001 de 24/08/2001. O documen...,COVID-19,não,0.0,0,0.84,293,2021-02-24,37647,15345716782
44,2021-02-24,HC,197921,15345716744,stf,HABEAS CORPUS 197.921 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.76,38,2020-04-02,183275,15342797616
195,2021-03-04,HC,198212,15345803466,stf,HABEAS CORPUS 198.212 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.71,38,2020-04-02,183275,15342797616
310,2021-03-10,HC,198482,15345858301,stf,HABEAS CORPUS 198.482 GOIÁS\n\nRELATOR : MIN. ...,COVID-19,não,0.0,0,0.73,38,2020-04-02,183275,15342797616
395,2021-03-15,HC,198517,15345907121,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 198.517 MINAS...,COVID-19,não,0.0,0,0.72,71,2020-04-16,183335,15342895143
396,2021-03-15,HC,198574,15345906396,stf,HABEAS CORPUS 198.574 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.73,38,2020-04-02,183275,15342797616
453,2021-03-17,HC,196383,15345928785,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 196.383 SÃO P...,COVID-19,não,0.0,0,0.77,449,2021-05-28,199764,15346541012


In [ ]:
#grupo_23['Data'] = pd.to_datetime(grupo_23['Data'])
#grupo_23['Data_SIM'] = pd.to_datetime(grupo_23['Data_SIM'])
#grupo_23 = grupo_23[grupo_23['Data'] > grupo_23['Data_SIM']]
#grupo_23


In [ ]:
# Usando o método map para colocar no dataframe (grupo_1) a coluna com as datas relacionadas ao indice do GRUPO_SIM
grupo_6['tipo_SIM'] =grupo_23['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Tipo'])
grupo_6

<ipython-input-41-e5984238470b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_6['tipo_SIM'] =grupo_23['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Tipo'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM,Número_SIM,Doc ID_SIM,tipo_SIM
22,2021-02-23,MS,37647,15345716782,stf,me MP n° 2.200-2/2001 de 24/08/2001. O documen...,COVID-19,não,0.0,0,0.84,293,2021-02-24,37647,15345716782,NaN
44,2021-02-24,HC,197921,15345716744,stf,HABEAS CORPUS 197.921 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.76,38,2020-04-02,183275,15342797616,NaN
195,2021-03-04,HC,198212,15345803466,stf,HABEAS CORPUS 198.212 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.71,38,2020-04-02,183275,15342797616,NaN
310,2021-03-10,HC,198482,15345858301,stf,HABEAS CORPUS 198.482 GOIÁS\n\nRELATOR : MIN. ...,COVID-19,não,0.0,0,0.73,38,2020-04-02,183275,15342797616,NaN
395,2021-03-15,HC,198517,15345907121,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 198.517 MINAS...,COVID-19,não,0.0,0,0.72,71,2020-04-16,183335,15342895143,NaN
396,2021-03-15,HC,198574,15345906396,stf,HABEAS CORPUS 198.574 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.73,38,2020-04-02,183275,15342797616,NaN
453,2021-03-17,HC,196383,15345928785,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 196.383 SÃO P...,COVID-19,não,0.0,0,0.77,449,2021-05-28,199764,15346541012,NaN


In [ ]:
nome_do_arquivo = 'grupo_6.xlsx'
grupo_6.to_excel(nome_do_arquivo, index=False)

# Local onde o arquivo é salvo no Google Colab
print("O arquivo foi salvo em:", nome_do_arquivo)


O arquivo foi salvo em: grupo_6.xlsx


In [ ]:
from google.colab import drive
import shutil

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o caminho para o arquivo Excel no sistema de arquivos temporário do Google Colab
caminho_arquivo_colab = 'grupo_6.xlsx'

# Definir o caminho para onde queremos mover o arquivo no Google Drive
caminho_destino_drive = '/content/drive/MyDrive/grupo_6.xlsx'

# Mover o arquivo para o Google Drive
shutil.move(caminho_arquivo_colab, caminho_destino_drive)

print("Arquivo movido com sucesso para o Google Drive em:", caminho_destino_drive)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Arquivo movido com sucesso para o Google Drive em: /content/drive/MyDrive/grupo_6.xlsx


grupo 7

In [ ]:
grupo_7.reset_index(drop=True, inplace=True)
grupo_7

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
0,2021-03-18,HC,198998,15345936905,stf,HABEAS CORPUS 198.998 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0,0
1,2021-03-18,HC,199005,15345946009,stf,HABEAS CORPUS 199.005 RIO GRANDE DO SUL\n\nREL...,COVID-19,não,0.0,0,0,0
2,2021-03-18,HC,199045,15345941017,stf,HABEAS CORPUS 199.045 SÃO PAULO\n\nRELATORA : ...,COVID-19,não,0.0,0,0,0
3,2021-03-18,HC,199055,15345934994,stf,HABEAS CORPUS 199.055 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0,0
4,2021-03-18,HC,199141,15345960153,stf,HABEAS CORPUS 199.141 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,2021-04-09,RHC,195415,15346118898,stf,Ementa e Acórdão\n\n22/03/2021 PRIMEIRA TURMA\...,COVID-19,não,0.0,0,0,0
496,2021-04-09,RHC,198185,15346124521,stf,Documento assinado digitalmente conforme MP n°...,COVID-19,não,0.0,0,0,0
497,2021-04-09,SS,5482,15346110701,stf,SUSPENSÃO DE SEGURANÇA 5.482 PARANÁ\n\nREGISTR...,COVID-19,não,0.0,0,0,0
498,2021-04-12,ADI,6625,15346127091,stf,Ementa e Acórdão\n\n08/03/2021 PLENÁRIO\n\nREF...,COVID-19,não,0.0,0,0,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
for i in range(len(grupo_7)):
  texto_a_ser_comparado =grupo_7["Conteudo"][i]
  #texto_a_ser_comparado =GRUPO_NAO.iloc[i]['Conteudo']
  P=[]
  #for j in range(len(df1111)):
  for j in range(len(GRUPO_SIM)):
    #valor = GRUPO_SIM.iloc[0]['Conteudo']
    #texto_fonte=GRUPO_SIM.iloc[j]['Conteudo']
    texto_fonte= GRUPO_SIM["Conteudo"][j]
    n=1
    #instancia o contador de n-grams
    counts=CountVectorizer(analyzer='word',ngram_range=(n,n))
    #cria um dicionario de n grams
    vocab2int=counts.fit([texto_a_ser_comparado,texto_fonte]).vocabulary_
    n_grams=counts.fit_transform([texto_a_ser_comparado,texto_fonte])
    #cria uma matriz de contagem para os dois textos
    n_grams_array = n_grams.toarray()
    #matriz de

    intersection_list= np.amin(n_grams.toarray(),axis=0)
    #intersection_list= np.amin(n_grams.toarray())
    intersection_list
   #contagem interseçao
    intersection_count= np.sum(intersection_list)
    index_A=0
    A_count=np.sum(n_grams.toarray()[index_A])
    p=intersection_count/A_count
    pt = round(p, 2)
    P.append(pt)
    #colocar o vetor e adicionar este p no vetor
  B=[]
  Z=[]
  probabilidade_maxima = max(P)
  localizaçao  = P.index(probabilidade_maxima)
  grupo_7['PROBABILIDADE'][i]=probabilidade_maxima
  grupo_7['RESULTADO'][i]=localizaçao
  #df2222['PROBABILIDADE'][i]=probabilidade_maxima
  #df2222['RESULTADO_df1'][i]=localizaçao

A saída de streaming foi truncada nas últimas 5000 linhas.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_7['PROBABILIDADE'][i]=probabilidade_maxima
<ipython-input-21-e1a38dac43e5>:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.42' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  grupo_7['PROBABILIDADE'][i]=probabilidade_maxima
<ipython-input-21-e1a38dac43e5>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_7['RESULTADO'][i]=localizaçao
<ipython-input-21-e1a38dac43e5>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in 

In [ ]:
GRUPO_SIM['Data'] = pd.to_datetime(GRUPO_SIM['Data'])
grupo_7['Data'] = pd.to_datetime(grupo_7['Data'])

<ipython-input-23-a7879a8a52d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_SIM['Data'] = pd.to_datetime(GRUPO_SIM['Data'])
<ipython-input-23-a7879a8a52d6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_7['Data'] = pd.to_datetime(grupo_7['Data'])


In [ ]:
condicao = grupo_7['PROBABILIDADE'] > 0.7

# Filtrar o DataFrame com base na condição
grupo_7 = grupo_7[condicao]

# Exibir o DataFrame filtrado
grupo_7

,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO
64,2021-03-22,HC,198991,15345973837,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 198.991 SÃO P...,COVID-19,não,0.0,0,0.89,524
146,2021-03-24,HC,198874,15346000506,stf,HABEAS CORPUS 198.874 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.82,38
188,2021-03-25,HC,199274,15346012346,stf,HABEAS CORPUS 199.274 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0.75,38
189,2021-03-25,HC,199371,15346006790,stf,HABEAS CORPUS 199.371 GOIÁS\n\nRELATOR : MIN. ...,COVID-19,não,0.0,0,0.75,457
272,2021-03-30,HC,199302,15346036621,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 199.302 SÃO P...,COVID-19,não,0.0,0,0.76,558
307,2021-04-05,HC,199173,15346050221,stf,HABEAS CORPUS 199.173 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0.71,38
346,2021-04-06,HC,199216,15346063528,stf,HABEAS CORPUS 199.216 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.79,38
348,2021-04-06,HC,199324,15346063476,stf,HABEAS CORPUS 199.324 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0.80,38
355,2021-04-06,HC,199595,15346064717,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 199.595 MINAS...,COVID-19,não,0.0,0,0.83,558
408,2021-04-07,HC,199668,15346091581,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 199.668 PARAN...,COVID-19,não,0.0,0,0.71,449


In [ ]:
#criando uma coluna repetindo o indice neste dataframe GRUPO_SIM
GRUPO_SIM['indice'] = GRUPO_SIM.index
GRUPO_SIM

<ipython-input-25-f7d753fe9fe5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GRUPO_SIM['indice'] = GRUPO_SIM.index


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,indice
0,2004-08-31,RE,199272,1627423,stf,DECISÃO: Atendo ao agravo e reconsidero a deci...,COVID-19,sim,0.01,3,0,0,0
1,2011-11-16,Rcl,2780,1605851,stf,DJe 05/12/2011\n\nEmenta e Acórdão\n\n16/11/20...,COVID-19,sim,0.01,3,0,0,1
2,2012-09-28,RE,701022,101824085,stf,RECURSO EXTRAORDINÁRIO 701.022 SANTA CATARINA\...,COVID-19,sim,0.01,3,0,0,2
3,2012-10-03,RE,701022,101824085,stf,RECURSO EXTRAORDINÁRIO 701.022 SANTA CATARINA\...,COVID-19,sim,0.01,3,0,0,3
4,2012-10-10,RE,542485,101445730,stf,AG.REG. NO RECURSO EXTRAORDINÁRIO 542.485 RIO ...,COVID-19,sim,0.01,3,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512,2008-05-29,AI,715019,2811652,stf,Trata-se de agravo de instrumento contra decis...,Zika vírus_Zika virus disease,sim,0.01,3,0,0,1512
1513,2008-06-09,AI,715019,2378326,stf,Trata-se de agravo de instrumento contra decis...,Zika vírus_Zika virus disease,sim,0.01,3,0,0,1513
1514,2009-03-09,RE,596025,3093042,stf,DECISÃO: 1. Trata-se de recurso extraordinário...,Zika vírus_Zika virus disease,sim,0.01,3,0,0,1514
1515,2009-03-17,RE,596025,2564402,stf,DECISÃO: 1. Trata-se de recurso extraordinário...,Zika vírus_Zika virus disease,sim,0.01,3,0,0,1515


In [ ]:
# Usando o método map para colocar no dataframe (grupo_1) a coluna com as datas relacionadas ao indice do GRUPO_SIM
grupo_7['Data_SIM'] =grupo_3['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Data'])
grupo_7

<ipython-input-26-2fadefffcf42>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_7['Data_SIM'] =grupo_3['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Data'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM
64,2021-03-22,HC,198991,15345973837,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 198.991 SÃO P...,COVID-19,não,0.0,0,0.89,524,NaT
146,2021-03-24,HC,198874,15346000506,stf,HABEAS CORPUS 198.874 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.82,38,NaT
188,2021-03-25,HC,199274,15346012346,stf,HABEAS CORPUS 199.274 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0.75,38,NaT
189,2021-03-25,HC,199371,15346006790,stf,HABEAS CORPUS 199.371 GOIÁS\n\nRELATOR : MIN. ...,COVID-19,não,0.0,0,0.75,457,NaT
272,2021-03-30,HC,199302,15346036621,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 199.302 SÃO P...,COVID-19,não,0.0,0,0.76,558,NaT
307,2021-04-05,HC,199173,15346050221,stf,HABEAS CORPUS 199.173 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0.71,38,NaT
346,2021-04-06,HC,199216,15346063528,stf,HABEAS CORPUS 199.216 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.79,38,NaT
348,2021-04-06,HC,199324,15346063476,stf,HABEAS CORPUS 199.324 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0.80,38,NaT
355,2021-04-06,HC,199595,15346064717,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 199.595 MINAS...,COVID-19,não,0.0,0,0.83,558,NaT
408,2021-04-07,HC,199668,15346091581,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 199.668 PARAN...,COVID-19,não,0.0,0,0.71,449,NaT


In [ ]:
grupo_7['Número_SIM'] =grupo_7['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Número'])
grupo_7['Doc ID_SIM'] =grupo_7['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Doc ID'])
grupo_7

<ipython-input-27-bf7d092e7440>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_7['Número_SIM'] =grupo_7['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Número'])
<ipython-input-27-bf7d092e7440>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_7['Doc ID_SIM'] =grupo_7['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Doc ID'])


,Data,Tipo,Número,Doc ID,Origem,Conteudo,Major Health Topics,OMS,PROBABILIDADE_OMS,RESULTADO_OMS,PROBABILIDADE,RESULTADO,Data_SIM,Número_SIM,Doc ID_SIM
64,2021-03-22,HC,198991,15345973837,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 198.991 SÃO P...,COVID-19,não,0.0,0,0.89,524,NaT,205073,15347303715
146,2021-03-24,HC,198874,15346000506,stf,HABEAS CORPUS 198.874 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.82,38,NaT,183275,15342797616
188,2021-03-25,HC,199274,15346012346,stf,HABEAS CORPUS 199.274 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0.75,38,NaT,183275,15342797616
189,2021-03-25,HC,199371,15346006790,stf,HABEAS CORPUS 199.371 GOIÁS\n\nRELATOR : MIN. ...,COVID-19,não,0.0,0,0.75,457,NaT,202594,15346616333
272,2021-03-30,HC,199302,15346036621,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 199.302 SÃO P...,COVID-19,não,0.0,0,0.76,558,NaT,207284,15348480805
307,2021-04-05,HC,199173,15346050221,stf,HABEAS CORPUS 199.173 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0.71,38,NaT,183275,15342797616
346,2021-04-06,HC,199216,15346063528,stf,HABEAS CORPUS 199.216 SÃO PAULO\n\nRELATOR : M...,COVID-19,não,0.0,0,0.79,38,NaT,183275,15342797616
348,2021-04-06,HC,199324,15346063476,stf,HABEAS CORPUS 199.324 MINAS GERAIS\n\nRELATOR ...,COVID-19,não,0.0,0,0.80,38,NaT,183275,15342797616
355,2021-04-06,HC,199595,15346064717,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 199.595 MINAS...,COVID-19,não,0.0,0,0.83,558,NaT,207284,15348480805
408,2021-04-07,HC,199668,15346091581,stf,MEDIDA CAUTELAR NO HABEAS CORPUS 199.668 PARAN...,COVID-19,não,0.0,0,0.71,449,NaT,199764,15346541012


In [ ]:
# Usando o método map para colocar no dataframe (grupo_1) a coluna com as datas relacionadas ao indice do GRUPO_SIM
#grupo_22['tipo_SIM'] =grupo_22['RESULTADO'].map(GRUPO_SIM.set_index('indice')['Tipo'])
#grupo_22_sem_datas=grupo_22
#grupo_22_sem_datas

In [ ]:
nome_do_arquivo = 'grupo_7.xlsx'
grupo_7.to_excel(nome_do_arquivo, index=False)

# Local onde o arquivo é salvo no Google Colab
print("O arquivo foi salvo em:", nome_do_arquivo)

O arquivo foi salvo em: grupo_7.xlsx


In [ ]:
from google.colab import drive
import shutil

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o caminho para o arquivo Excel no sistema de arquivos temporário do Google Colab
caminho_arquivo_colab = 'grupo_7.xlsx'

# Definir o caminho para onde queremos mover o arquivo no Google Drive
caminho_destino_drive = '/content/drive/MyDrive/grupo_7.xlsx'

# Mover o arquivo para o Google Drive
shutil.move(caminho_arquivo_colab, caminho_destino_drive)

print("Arquivo movido com sucesso para o Google Drive em:", caminho_destino_drive)

Mounted at /content/drive
Arquivo movido com sucesso para o Google Drive em: /content/drive/MyDrive/grupo_7.xlsx
